# Test delensalots `sims.sims_lib` module
This notebook serves in understanding how data can be provided and accessed and we see the behaviour of the libraries

In [1]:
from IPython.display import clear_output
from datetime import datetime
import os
from os.path import join as opj

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

import delensalot
from delensalot.utility.utils_hp import gauss_beam
from delensalot.sims.sims_lib import Xunl, Xsky, Xobs, Simhandler

os.environ["SCRATCH"] = "./SCRATCH/delensalot"

cond4mpi does not exists. No MPI loaded
mpi.py: Plancklens.mpi disabled as per environ variable 



# Main interface
is `Simhandler`, and provides the most important function `get_sim_pmap()` which returns the observed QU maps. Simhandler is flexible: you can provide `cls` to generate realizations from, (un)lensed realizations, or observed maps. Simhandler will do the rest for you.

The two most important parameter are `flavour` and `space`. They are used to decide how to work with the data.
 * flavour can be in [`unl`,`sky`,`obs`],
 * space in [`cl`,`alm`,`map`]

`Simhandler` never stores the generated data on disk per default, but in memory.

## flavour = `obs`
 1. already on disk
 2. need to be generated

For the first case, simply provide `libdir`, `fns`, `spin`, `lmax`, so delensalot knows exactly what this data is. 

In [2]:
sims = Simhandler(
    flavour='obs', space='map', 
    libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})",
    fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'],
    spin=2,
    lmax=4096)
sims.get_sim_obs(0, space='map', field='polarization', spin=2)

array([[ 16.93667809, -10.69699026,  11.10045604, ...,  -2.31878458,
          3.82031646,  -0.31462867],
       [  5.91189672,   0.10350747,  -0.43444732, ...,   2.28714553,
         -6.70083286,   9.01127998]])

In the second case, there is no obs data, and `get_sim_obs()` will therefore fail. This is intended. We don't accidently want to generate fake data being generated.

In [3]:
sims = Simhandler(
    flavour='obs', space='map')
sims.get_sim_obs(0, space='map', field='polarization', spin=2)

AssertionError: need to provide libdir

## flavour = `sky`
 1. already on disk
 2. need to be generated

For the first case, simply provide `libdir`, `fns`, `spin`, `lmax`, so delensalot knows exactly what this data is. Additionally, to generate the observed maps, it needs a `transfunction`, and `nlev`.

In [2]:
sims = Simhandler(
    flavour='sky', space='map', 
    libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})",
    fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'],
    spin=2,
    lmax=4096,
    transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096),
    nlev={'P': np.sqrt(10)},
    )
obs = sims.get_sim_obs(0, space='map', field='polarization', spin=2)

For the second case, again, it will fail as intended.

In [6]:
sims = Simhandler(
    flavour='unl', space='map',
    spin=2,
    lmax=4096,
    nlev={'P': np.sqrt(10)},
    transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096),
    )
obs = sims.get_sim_obs(0, space='map', field='polarization', spin=2)

AssertionError: need to provide libdir

## flavour = `unl`
 1. already on disk
 2. need to be generated

For the first case, simply provide `libdir`, `fns`, `spin`, `lmax`, so delensalot knows exactly what this data is. Additionally, to generate the lensed maps, you can provide a phi realization yourself. Again, it needs a `transfunction`, and `nlev`.

In [3]:
sims = Simhandler(
    flavour='unl', space='map', 
    libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})",
    fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'],
    libdir_phi="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})",
    fnsP='Qmapobs_{}.npy',
    phi_field='potential',
    phi_space='map',
    spin=2,
    lmax=4096,
    phi_lmax=4096,
    transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096),
    nlev={'P': np.sqrt(10)},
    )
obs = sims.get_sim_obs(0, space='map', field='polarization', spin=2)

deflection std is 1.92e+07 amin: this is really too high a value for something sensible


It can happen that you are provided unlensed maps, and only the phi power spectrum. You can generate observed maps from this as follows

In [ ]:
sims = Simhandler(
    flavour='unl', space='map', 
    libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})",
    fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'],
    spin=2,
    lmax=4096,
    phi_field='potential',
    phi_space='cl',
    phi_fn=opj(os.path.dirname(delensalot.__file__), 'data', 'cls', 'FFP10_wdipole_lenspotentialCls.dat'),
    phi_lmax=4096,
    transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096),
    nlev={'P': np.sqrt(10)},
    )
obs = sims.get_sim_obs(0, space='map', field='polarization', spin=2)

## flavour = `unl`, space= `cl`
 1. need to be generated

Simply provide `lmax` and `phi_lmax`, so delensalot knows exactly what this data is. You can provide two power spectra, one for the CMB, and one for phi, but you can also leave it blank. Again, it needs a `transfunction`, and `nlev`.

In [ ]:
sims = Simhandler(
    flavour='unl', space='cl',
    lmax=4096,
    phi_lmax=4096,
    CMB_fn = opj(os.path.dirname(delensalot.__file__), 'data', 'cls', 'FFP10_wdipole_lenspotentialCls.dat'),
    phi_fn = opj(os.path.dirname(delensalot.__file__), 'data', 'cls', 'FFP10_wdipole_lenspotentialCls.dat'),
    phi_field = 'potential',
    phi_space = 'cl',
    transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096),
    nlev={'P': np.sqrt(10)},
    )
obs = sims.get_sim_obs(0, space='map', field='polarization', spin=2)

In [ ]:
sims = Simhandler(
    flavour='unl', space='cl',
    lmax=4096,
    phi_lmax=4096,
    CMB_fn = opj(os.path.dirname(delensalot.__file__), 'data', 'cls', 'FFP10_wdipole_lenspotentialCls.dat'),
    transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096),
    nlev={'P': np.sqrt(10)},
    )
obs = sims.get_sim_obs(0, space='map', field='polarization', spin=2)

# Individual libraries
Tests for
 * `class Cls()`
 * `class Xunl()`
 * `class Xsky()`
 * `class Xobs()`

In [2]:
synobs = Xobs(lmax=4096, transfunction=gauss_beam(1.0/180/60 * np.pi, lmax=4096), nlev={'P': np.sqrt(10)}, geometry=('healpix', {'nside': 2048}))
synobs.get_sim_obs(0, field='polarization', space='map', spin=2)

In [5]:
synobs_prec = Xobs(lmax=4096, libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})", fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'], space='map')
synobs_prec.get_sim_obs(0, field='polarization', space='map', spin=2)

array([[ 16.93667809, -10.69699026,  11.10045604, ...,  -2.31878458,
          3.82031646,  -0.31462867],
       [  5.91189672,   0.10350747,  -0.43444732, ...,   2.28714553,
         -6.70083286,   9.01127998]])

In [7]:
synsky = Xsky(lmax=4096)
synsky.get_sim_sky(0, field='polarization', space='map', spin=2)


In [3]:
synsky_prec = Xsky(lmax=4096, libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})", fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'], space='map', spin=2)
synsky_prec.get_sim_sky(0, field='polarization', space='map', spin=2)

array([[ 16.93667809, -10.69699026,  11.10045604, ...,  -2.31878458,
          3.82031646,  -0.31462867],
       [  5.91189672,   0.10350747,  -0.43444732, ...,   2.28714553,
         -6.70083286,   9.01127998]])

In [9]:
synunl = Xunl(lmax=4096)
synunl.get_sim_unl(0, field='polarization', space='map', spin=2)


In [4]:
synunl_prec = Xunl(lmax=4096, libdir="/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/notebooks/dlensalot/dev/SCRATCH/delensalot/sims/('healpix', {'nside': 2048})", fns=['Qmapobs_{}.npy', 'Umapobs_{}.npy'], space='map', spin=2)
synunl_prec.get_sim_unl(0, field='polarization', space='map', spin=2)

array([[ 16.93667809, -10.69699026,  11.10045604, ...,  -2.31878458,
          3.82031646,  -0.31462867],
       [  5.91189672,   0.10350747,  -0.43444732, ...,   2.28714553,
         -6.70083286,   9.01127998]])